for legal tokenization and NER

In [5]:
!pip install -U ckiptagger tensorflow==1.15 --user

Requirement already up-to-date: tensorflow==1.15 in c:\programdata\anaconda3\lib\site-packages (1.15.0)


In [1]:

from ckiptagger import construct_dictionary, WS, POS, NER
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.feature_extraction.text import *
# from summa import keywords
import time
import tensorflow as tf
tf.__version__

'1.15.0'

initialize

In [2]:
define_dict_path = "./TokSentLeo/user_dict/company_dict.txt"
model_path = './TokSentLeo/CKIP_model/'
doc_path = "D:\\python\\LEO_TM\\BERT_chinese_LM_processing\\Data\\NSL_POC\\paragraph.csv"
ws = WS(model_path)
pos = POS(model_path)
ner = NER(model_path)

In [3]:
root_dir2 = "../Data/NSL_POC/"
df2 = pd.read_csv(root_dir2+"paragraph.csv",encoding='cp950')
df2

,index,content
0,1,法規名稱：中華民國證券投資信託暨顧問商業同業公會「會員及其銷售機構從事廣告及營業活動行為規範...
1,2,第一章 總則\n第一條\n本行為規範依據證券投資顧問事業管理規則第十四條第二項、證券投資...
2,3,第二條\n本公會會員及其銷售機構，為經營、辦理或推展證券投資信託、證券投資\n顧問、全權委託...
3,4,第三條\n證券投資信託事業委任之基金銷售機構從事證券投資信託基金之廣告、公\n開說明會及其他...
4,5,第四條\n總代理人委任之基金銷售機構從事境外基金之廣告、公開說明會及其他營\n業活動有違反本...
5,6,第五條\n本行為規範所稱廣告，指以促進業務為目的，運用下列傳播媒體，就第二\n條業務及其相關...
6,7,第六條\n本行為規範所稱公開說明會及其他營業活動，指為促進業務為目的，以舉\n辦現場講習會、...
7,8,第二章 證券投資信託暨境外基金業務\n第七條\n證券投資信託事業、總代理人及其委任之基金...
8,9,第八條\n證券投資信託事業、總代理人及基金銷售機構從事基金之廣告、公開說明\n會及其他營業活...
9,10,第八條之一\n證券投資信託事業、總代理人及基金銷售機構從事基金業務之宣導推廣活\n動時，得在...


preprocess

In [4]:
def convert_content(s):
    """
    split a paragraph by "。"
    """
    paragraph = "".join(s.split("\n"))
    paragraph = paragraph.replace("\n",'')
    return paragraph

user dictionary

In [5]:
word_to_weight = {}
with open(define_dict_path, "r", encoding='utf8') as file:
    for line in file:
        key, value = line.split()
        word_to_weight[str(key)] = 2
dictionary = construct_dictionary(word_to_weight)

In [6]:
df2['content'] = df2.content.map(convert_content)
pretrain_doc2_df = df2.filter(["content"])
pretrain_doc2_df = pretrain_doc2_df.reset_index(drop=True)
pretrain_doc2_df

,content
0,法規名稱：中華民國證券投資信託暨顧問商業同業公會「會員及其銷售機構從事廣告及營業活動行為規範...
1,第一章 總則第一條本行為規範依據證券投資顧問事業管理規則第十四條第二項、證券投資信託事業...
2,第二條本公會會員及其銷售機構，為經營、辦理或推展證券投資信託、證券投資顧問、全權委託投資與境...
3,第三條證券投資信託事業委任之基金銷售機構從事證券投資信託基金之廣告、公開說明會及其他營業活動...
4,第四條總代理人委任之基金銷售機構從事境外基金之廣告、公開說明會及其他營業活動有違反本行為規範...
5,第五條本行為規範所稱廣告，指以促進業務為目的，運用下列傳播媒體，就第二條業務及其相關事務為傳...
6,第六條本行為規範所稱公開說明會及其他營業活動，指為促進業務為目的，以舉辦現場講習會、座談會、...
7,第二章 證券投資信託暨境外基金業務第七條證券投資信託事業、總代理人及其委任之基金銷售機構...
8,第八條證券投資信託事業、總代理人及基金銷售機構從事基金之廣告、公開說明會及其他營業活動時，不...
9,第八條之一證券投資信託事業、總代理人及基金銷售機構從事基金業務之宣導推廣活動時，得在不與基金...


In [10]:
%%time
all_sent_li = pretrain_doc2_df.content.tolist()
word_sentence_list = ws(all_sent_li, recommend_dictionary=dictionary,
                        segment_delimiter_set={",", "。", ":", "?", "!", ";", "，", "：", "？", "！", "；"})
pos_sentence_list = pos(word_sentence_list)
entity_sentence_list = ner(word_sentence_list, pos_sentence_list)
# temp = []
# bad_list = ["<br>", "br", "BR", "<BR>", "，", "【", "】", "╱", "▲", "▼", "&amp;amp;amp;amp;amp;lt;br",
#             "&amp;amp;amp;amp;amp;gt", "amp", "lt", "br&", "gt", "&amp", "[", "]"]
# for w_s_l in word_sentence_list:
#     t = [x for x in w_s_l if x not in bad_list]
#     temp.append(t)
# word_sentence_list = temp
print(word_sentence_list[0])

TypeError: 'str' object is not callable

In [11]:
# statistics
all_kind_pos = []
for li in pos_sentence_list:
    for po in li:
        all_kind_pos.append(po)
all_kind_pos = list(set(all_kind_pos))
all_kind_pos = sorted(all_kind_pos)
# pos_dict.keys = all_kind_pos
print(len(all_kind_pos))

51


In [22]:
# pos_dict = {}
# for pos in all_kind_pos:
#     pos_dict[pos] = []
pos_df = pd.DataFrame(columns=all_kind_pos)
for i,(word_sentence, pos_sentence) in enumerate(zip(word_sentence_list, pos_sentence_list)):
#     print(i)
    assert len(word_sentence) == len(pos_sentence)
    pos_dict = {}
    for pos in all_kind_pos:
        pos_dict[pos] = []
    for word, pos in zip(word_sentence, pos_sentence):
        temp = pos_dict[pos]
        temp.append(word)
        pos_dict[pos] = temp

    temp = pd.DataFrame.from_dict([pos_dict])
    pos_df = pos_df.append(temp)
pos_df = pos_df.reset_index(drop=True)
pos_df

,A,COLONCATEGORY,COMMACATEGORY,Caa,Cab,Cbb,D,DE,DOTCATEGORY,Da,...,VF,VG,VH,VHC,VI,VJ,VK,VL,V_2,WHITESPACE
0,[信託],"[：, ：]",[],"[暨, 及, 及]",[],[],[],[],[],[],...,[],[],[],[],[],[從事],[],[],[],[]
1,"[信託, 信託]",[],[],"[暨, 及]",[],[],[],[之],[],[],...,[],[本行為],[],[],[],[],[],[],[],[ ]
2,"[信託, 信託]",[],"[，, ，, ，, ，]","[及, 或, 與, 及, 及, 及]",[],[並],"[全權, 應]",[],[],[],...,[委託],[],"[相關, 相關, 專業]",[公開],[],"[從事, 恪遵]",[],[],[],[]
3,"[信託, 信託, 信託]",[],"[，, ，]",[及],[],[],"[確實, 不可, 應]","[之, 之, 之]",[],[],...,[委任],[],[相關],[公開],[歸責],"[從事, 違反, 負]",[],[],"[有, 有]",[]
4,[],[],"[，, ，, ，, ，, ，]","[及, 及, 或, 及, 及]",[],[但],"[應, 得, 確實, 不可, 應]","[之, 之, 之, 之, 之]",[],[],...,"[委任, 委任, 委任]",[],"[相關, 相關, 相關]",[公開],[歸責],"[從事, 違反, 負, 違反, 負]",[],[],"[有, 有, 有]",[]
5,"[下列, 公共]",[：],"[，, ，, ，, ，, ，]","[及, 或, 或, 或, 或, 或, 或, 與, 或, 或, 或, 或]","[等, 等, 等]",[],"[所, 所, 所]","[之, 之, 之, 之, 之, 之, 之, 之]",[],[],...,[],"[本行為, 稱, 指, 為, 為, 稱]","[相關, 靜止, 相關]","[公開, 公開]",[],"[有關, 載有]",[促進],[],[],"[ , , ]"
6,"[一般, 下列]",[：],"[，, ，, ，, ，, ，, ，, ，, ，, ，, ，]","[及, 或, 或, 以及, 或, 或, 或, 及, 及, 及, 及, 及, 或, 或, 或,...",[等],"[一, 並, 四]","[所, 應, 應, 所, 不得, 所, 應, 應, 得, 應]","[之, 之, 之, 之, 之, 之, 之, 之, 之, 之]",[],[],...,[],"[本行為, 稱, 指為, 為]","[誠實, 真實, 衡平, 顯著, 淺顯, 易懂, 必要]","[公開, 公開, 公開, 充實]",[],"[有關, 從事, 遵守, 低於]","[促進, 涉及]",[致力],[],"[ , ]"
7,"[信託, 信託, 下列]",[：],"[，, ，, ，, ，, ，, ，, ，, ，, ，, ，, ，]","[暨, 及, 或, 或, 及, 及]",[等],"[以, 並, 一, 如, 但, 四, 並, 若]","[須, 應, 所, 應, 應, 則, 免, 仍, 應, 應, 應, 確實, 再, 應, 所,...","[之, 之, 之, 之, 之, 之, 之, 之, 得, 之, 之, 之, 之]",[],[],...,"[委任, 申請]",[簡稱],"[相關, 新, 專業, 相關, 不實]",[公開],"[為限, 為主]","[有關, 遵守, 符合, 從事]",[涉及],[],[],"[ , , , , , ]"
8,"[信託, 下列, 信託, 主要, 開放式, 主要, 主要, 淨, 主要]",[：],"[，, ，, ，, ，, ，, ，, ，, ，, ，, ，, ，, ，, ，, ，, ，, ...","[及, 及, 或, 或, 或, 或, 或, 或, 及, 或, 或, 或, 或, 或, 及, ...","[等, 等, 等, 等, 等, 等]","[一, 但, 但, 四, 六, 十二, 以, 十八, 但, 以免, 若]","[不得, 已, 能, 已, 未, 預, 可能, 不, 未, 未, 所, 已, 尚未, 未, ...","[之, 之, 之, 之, 之, 之, 之, 之, 之, 之, 之, 之, 之, 之, 之, ...",[],[約],...,"[申請, 勸誘, 促, 供, 供, 供, 供]","[作為, 為, 作為, 為, 作為, 為, 為, 作為, 為, 作為, 為, 為, 為, 為...","[生效, 安全, 獲利, 充分, 具體, 安全, 不在此限, 不在此限, 虛偽, 生效, 相...","[公開, 公開, 公開, 公開, 平衡, 公開, 公開, 公開, 混淆]",[],"[從事, 誤, 設有, 違反, 貶低, 載有, 不符, 無, 折價, 優於, 有關, 獲, ...","[誤信, 誤信, 涉及, 載明, 反映, 同意, 載明]",[使],"[有, 有, 有]","[ , , , , , , , , , ..."
9,"[信託, 下列, 單一, 信託, 下列, 單一, 非, 信託, 信託]","[：, ：, ：]","[，, ，, ，, ，, ，, ，, ，, ，, ，, ，, ，, ，, ，, ，, ，, ...","[及, 或, 及, 或, 且, 或, 及, 及, 或, 或, 或]","[等, 等, 等]","[並, 並, 且, 一, 以, 但, 並, 四, 如, 並]","[得, 不, 應, 不得, 變相, 應, 不得, 重複, 不得, 應, 確實, 應, 應, ...","[之, 之, 之, 之, 之, 之, 之, 之, 之, 之, 之]",[],[僅],...,"[鼓勵, 鼓勵, 供, 交由]","[為, 作為, 類似]","[相關, 浮濫, 合理, 相關, 相關, 不在此限, 保守]",[結合],[],"[從事, 遵守, 維持, 含, 訂有, 低於, 印有]","[注意, 載明]",[流於],[有],"[ , , , , , ]"


In [51]:
cols = pos_df.columns.tolist()
cols = [EnZh_dict[x] for x in cols]
pos_df.columns = cols
pos_df

,非謂形容詞,冒號,逗號,對等連接詞,連接詞，如：等等,關聯連接詞,副詞,的之得地,點號,數量副詞,...,動作謂賓動詞,分類動詞,狀態不及物動詞,狀態使動動詞,狀態類及物動詞,狀態及物動詞,狀態句賓動詞,狀態謂賓動詞,有,空白
0,[信託],"[：, ：]",[],"[暨, 及, 及]",[],[],[],[],[],[],...,[],[],[],[],[],[從事],[],[],[],[]
1,"[信託, 信託]",[],[],"[暨, 及]",[],[],[],[之],[],[],...,[],[本行為],[],[],[],[],[],[],[],[ ]
2,"[信託, 信託]",[],"[，, ，, ，, ，]","[及, 或, 與, 及, 及, 及]",[],[並],"[全權, 應]",[],[],[],...,[委託],[],"[相關, 相關, 專業]",[公開],[],"[從事, 恪遵]",[],[],[],[]
3,"[信託, 信託, 信託]",[],"[，, ，]",[及],[],[],"[確實, 不可, 應]","[之, 之, 之]",[],[],...,[委任],[],[相關],[公開],[歸責],"[從事, 違反, 負]",[],[],"[有, 有]",[]
4,[],[],"[，, ，, ，, ，, ，]","[及, 及, 或, 及, 及]",[],[但],"[應, 得, 確實, 不可, 應]","[之, 之, 之, 之, 之]",[],[],...,"[委任, 委任, 委任]",[],"[相關, 相關, 相關]",[公開],[歸責],"[從事, 違反, 負, 違反, 負]",[],[],"[有, 有, 有]",[]
5,"[下列, 公共]",[：],"[，, ，, ，, ，, ，]","[及, 或, 或, 或, 或, 或, 或, 與, 或, 或, 或, 或]","[等, 等, 等]",[],"[所, 所, 所]","[之, 之, 之, 之, 之, 之, 之, 之]",[],[],...,[],"[本行為, 稱, 指, 為, 為, 稱]","[相關, 靜止, 相關]","[公開, 公開]",[],"[有關, 載有]",[促進],[],[],"[ , , ]"
6,"[一般, 下列]",[：],"[，, ，, ，, ，, ，, ，, ，, ，, ，, ，]","[及, 或, 或, 以及, 或, 或, 或, 及, 及, 及, 及, 及, 或, 或, 或,...",[等],"[一, 並, 四]","[所, 應, 應, 所, 不得, 所, 應, 應, 得, 應]","[之, 之, 之, 之, 之, 之, 之, 之, 之, 之]",[],[],...,[],"[本行為, 稱, 指為, 為]","[誠實, 真實, 衡平, 顯著, 淺顯, 易懂, 必要]","[公開, 公開, 公開, 充實]",[],"[有關, 從事, 遵守, 低於]","[促進, 涉及]",[致力],[],"[ , ]"
7,"[信託, 信託, 下列]",[：],"[，, ，, ，, ，, ，, ，, ，, ，, ，, ，, ，]","[暨, 及, 或, 或, 及, 及]",[等],"[以, 並, 一, 如, 但, 四, 並, 若]","[須, 應, 所, 應, 應, 則, 免, 仍, 應, 應, 應, 確實, 再, 應, 所,...","[之, 之, 之, 之, 之, 之, 之, 之, 得, 之, 之, 之, 之]",[],[],...,"[委任, 申請]",[簡稱],"[相關, 新, 專業, 相關, 不實]",[公開],"[為限, 為主]","[有關, 遵守, 符合, 從事]",[涉及],[],[],"[ , , , , , ]"
8,"[信託, 下列, 信託, 主要, 開放式, 主要, 主要, 淨, 主要]",[：],"[，, ，, ，, ，, ，, ，, ，, ，, ，, ，, ，, ，, ，, ，, ，, ...","[及, 及, 或, 或, 或, 或, 或, 或, 及, 或, 或, 或, 或, 或, 及, ...","[等, 等, 等, 等, 等, 等]","[一, 但, 但, 四, 六, 十二, 以, 十八, 但, 以免, 若]","[不得, 已, 能, 已, 未, 預, 可能, 不, 未, 未, 所, 已, 尚未, 未, ...","[之, 之, 之, 之, 之, 之, 之, 之, 之, 之, 之, 之, 之, 之, 之, ...",[],[約],...,"[申請, 勸誘, 促, 供, 供, 供, 供]","[作為, 為, 作為, 為, 作為, 為, 為, 作為, 為, 作為, 為, 為, 為, 為...","[生效, 安全, 獲利, 充分, 具體, 安全, 不在此限, 不在此限, 虛偽, 生效, 相...","[公開, 公開, 公開, 公開, 平衡, 公開, 公開, 公開, 混淆]",[],"[從事, 誤, 設有, 違反, 貶低, 載有, 不符, 無, 折價, 優於, 有關, 獲, ...","[誤信, 誤信, 涉及, 載明, 反映, 同意, 載明]",[使],"[有, 有, 有]","[ , , , , , , , , , ..."
9,"[信託, 下列, 單一, 信託, 下列, 單一, 非, 信託, 信託]","[：, ：, ：]","[，, ，, ，, ，, ，, ，, ，, ，, ，, ，, ，, ，, ，, ，, ，, ...","[及, 或, 及, 或, 且, 或, 及, 及, 或, 或, 或]","[等, 等, 等]","[並, 並, 且, 一, 以, 但, 並, 四, 如, 並]","[得, 不, 應, 不得, 變相, 應, 不得, 重複, 不得, 應, 確實, 應, 應, ...","[之, 之, 之, 之, 之, 之, 之, 之, 之, 之, 之]",[],[僅],...,"[鼓勵, 鼓勵, 供, 交由]","[為, 作為, 類似]","[相關, 浮濫, 合理, 相關, 相關, 不在此限, 保守]",[結合],[],"[從事, 遵守, 維持, 含, 訂有, 低於, 印有]","[注意, 載明]",[流於],[有],"[ , , , , , ]"


In [56]:
def combine_fn(df):
    '''
    input: dataframe
    return: 、
    '''
    return "、".join(df)

In [34]:
all_ner = []
for ner_doc in entity_sentence_list:
    for nerr in list(ner_doc):
        all_ner.append(nerr[2])
#     list(ner)[0][2]
all_ner = sorted(list(set(all_ner)))
all_ner

['CARDINAL',
 'DATE',
 'GPE',
 'LANGUAGE',
 'LAW',
 'MONEY',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'TIME',
 'WORK_OF_ART']

In [38]:
ner_df = pd.DataFrame(columns=all_ner)
for i,ner_doc in enumerate(entity_sentence_list):
    ner_dict = {}
    for nerr in all_ner:
        ner_dict[nerr] = []
    for nerr in list(ner_doc):
        temp = ner_dict[nerr[2]]
        temp.append(nerr[-1])
        ner_dict[nerr[2]] = temp

    temp = pd.DataFrame.from_dict([ner_dict])
    ner_df = ner_df.append(temp)
ner_df = ner_df.reset_index(drop=True)
ner_df

,CARDINAL,DATE,GPE,LANGUAGE,LAW,MONEY,ORDINAL,ORG,PERCENT,TIME,WORK_OF_ART
0,[],[民國 109 年 01 月 07 日],[],[],[],[],[],[中華民國證券投資信託暨顧問商業同業公會],[],[],[]
1,[],[],[],[],[],[],"[第二, 第二十二條, 第六, 第七, 第一, 第十四條, 第二, 第五十條, 第一]",[境外基金],[],[],[]
2,[],[],[],[],[],[],[第二],[],[],[],[]
3,[],[],[],[],[],[],[第三],[],[],[],[]
4,[],[],[],[],[],[],[第四],[],[],[],[]
5,"[二, 四, 三, 一, 六, 七, 五]",[],[],[],[],[],"[第五, 第二]",[],[],[],[]
6,"[四, 一, 三, 二]",[],[],[中文],[],[],"[第一, 第六]",[],[],[],[]
7,"[四, 二, 三, 一]",[],[],[],[],[],"[第二, 第七]","[金管會, 金管會]",[],[],[]
8,"[三十一, 十三, 十四, 二十二, 二十四, 二十八, 二十九, 七, 六, 三十三, 二...",[三十],[],[],[],[],[第八],"[金管會, 金管會, 金管會, 金管會, 金管會]",[],[],[基金月報]
9,"[二, 四, 五, 一, 一, 二, 四, 四, 四, 三, 三]","[上個月份, 上個月份, 二年, 每月十日前]",[],[],[],"[新臺幣二百元, 新臺幣三十元]","[第一, 第二, 第四, 第八]",[金管會],[],[],[]


In [54]:
cols = ner_df.columns.tolist()
cols = [EnZh_dict_ner[x] for x in cols]
ner_df.columns = cols
ner_df

,數字,日期,行政區,語言,法律,金錢,序數,組織,百分比率,時間,作品
0,[],[民國 109 年 01 月 07 日],[],[],[],[],[],[中華民國證券投資信託暨顧問商業同業公會],[],[],[]
1,[],[],[],[],[],[],"[第二, 第二十二條, 第六, 第七, 第一, 第十四條, 第二, 第五十條, 第一]",[境外基金],[],[],[]
2,[],[],[],[],[],[],[第二],[],[],[],[]
3,[],[],[],[],[],[],[第三],[],[],[],[]
4,[],[],[],[],[],[],[第四],[],[],[],[]
5,"[二, 四, 三, 一, 六, 七, 五]",[],[],[],[],[],"[第五, 第二]",[],[],[],[]
6,"[四, 一, 三, 二]",[],[],[中文],[],[],"[第一, 第六]",[],[],[],[]
7,"[四, 二, 三, 一]",[],[],[],[],[],"[第二, 第七]","[金管會, 金管會]",[],[],[]
8,"[三十一, 十三, 十四, 二十二, 二十四, 二十八, 二十九, 七, 六, 三十三, 二...",[三十],[],[],[],[],[第八],"[金管會, 金管會, 金管會, 金管會, 金管會]",[],[],[基金月報]
9,"[二, 四, 五, 一, 一, 二, 四, 四, 四, 三, 三]","[上個月份, 上個月份, 二年, 每月十日前]",[],[],[],"[新臺幣二百元, 新臺幣三十元]","[第一, 第二, 第四, 第八]",[金管會],[],[],[]


In [50]:
all_pos = '''A	非謂形容詞
Caa	對等連接詞
Cab	連接詞，如：等等
Cba	連接詞，如：的話
Cbb	關聯連接詞
D	副詞
Da	數量副詞
Dfa	動詞前程度副詞
Dfb	動詞後程度副詞
Di	時態標記
Dk	句副詞
DM	定量式
I	感嘆詞
Na	普通名詞
Nb	專有名詞
Nc	地方詞
Ncd	位置詞
Nd	時間詞
Nep	指代定詞
Neqa	數量定詞
Neqb	後置數量定詞
Nes	特指定詞
Neu	數詞定詞
Nf	量詞
Ng	後置詞
Nh	代名詞
Nv	名物化動詞
P	介詞
T	語助詞
VA	動作不及物動詞
VAC	動作使動動詞
VB	動作類及物動詞
VC	動作及物動詞
VCL	動作接地方賓語動詞
VD	雙賓動詞
VF	動作謂賓動詞
VE	動作句賓動詞
VG	分類動詞
VH	狀態不及物動詞
VHC	狀態使動動詞
VI	狀態類及物動詞
VJ	狀態及物動詞
VK	狀態句賓動詞
VL	狀態謂賓動詞
V_2	有
DE	的之得地
SHI	是
FW	外文
COLONCATEGORY	冒號
COMMACATEGORY	逗號
DASHCATEGORY	破折號
DOTCATEGORY	點號
ETCCATEGORY	刪節號
EXCLAMATIONCATEGORY	驚嘆號
PARENTHESISCATEGORY	括號
PAUSECATEGORY	頓號
PERIODCATEGORY	句號
QUESTIONCATEGORY	問號
SEMICOLONCATEGORY	分號
SPCHANGECATEGORY	雙直線
WHITESPACE	空白'''
all_pos = all_pos.split('\n')
EnZh_dict = {}
for ench in all_pos:
    cut = ench.split('\t')
    en = cut[0]
    ch = cut[1]
    EnZh_dict[en] = ch
EnZh_dict

{'A': '非謂形容詞',
 'Caa': '對等連接詞',
 'Cab': '連接詞，如：等等',
 'Cba': '連接詞，如：的話',
 'Cbb': '關聯連接詞',
 'D': '副詞',
 'Da': '數量副詞',
 'Dfa': '動詞前程度副詞',
 'Dfb': '動詞後程度副詞',
 'Di': '時態標記',
 'Dk': '句副詞',
 'DM': '定量式',
 'I': '感嘆詞',
 'Na': '普通名詞',
 'Nb': '專有名詞',
 'Nc': '地方詞',
 'Ncd': '位置詞',
 'Nd': '時間詞',
 'Nep': '指代定詞',
 'Neqa': '數量定詞',
 'Neqb': '後置數量定詞',
 'Nes': '特指定詞',
 'Neu': '數詞定詞',
 'Nf': '量詞',
 'Ng': '後置詞',
 'Nh': '代名詞',
 'Nv': '名物化動詞',
 'P': '介詞',
 'T': '語助詞',
 'VA': '動作不及物動詞',
 'VAC': '動作使動動詞',
 'VB': '動作類及物動詞',
 'VC': '動作及物動詞',
 'VCL': '動作接地方賓語動詞',
 'VD': '雙賓動詞',
 'VF': '動作謂賓動詞',
 'VE': '動作句賓動詞',
 'VG': '分類動詞',
 'VH': '狀態不及物動詞',
 'VHC': '狀態使動動詞',
 'VI': '狀態類及物動詞',
 'VJ': '狀態及物動詞',
 'VK': '狀態句賓動詞',
 'VL': '狀態謂賓動詞',
 'V_2': '有',
 'DE': '的之得地',
 'SHI': '是',
 'FW': '外文',
 'COLONCATEGORY': '冒號',
 'COMMACATEGORY': '逗號',
 'DASHCATEGORY': '破折號',
 'DOTCATEGORY': '點號',
 'ETCCATEGORY': '刪節號',
 'EXCLAMATIONCATEGORY': '驚嘆號',
 'PARENTHESISCATEGORY': '括號',
 'PAUSECATEGORY': '頓號',
 'PERIODCATEGORY': '句號',
 'QUESTIO

In [53]:
all_ner = '''CARDINAL	數字
DATE	日期
EVENT	事件
FAC	設施
GPE	行政區
LANGUAGE	語言
LAW	法律
LOC	地理區
MONEY	金錢
NORP	民族、宗教、政治團體
ORDINAL	序數
ORG	組織
PERCENT	百分比率
PERSON	人物
PRODUCT	產品
QUANTITY	數量
TIME	時間
WORK_OF_ART	作品'''
all_ner = all_ner.split('\n')
EnZh_dict_ner = {}
for ench in all_ner:
    cut = ench.split('\t')
    en = cut[0]
    ch = cut[1]
    EnZh_dict_ner[en] = ch
EnZh_dict_ner

{'CARDINAL': '數字',
 'DATE': '日期',
 'EVENT': '事件',
 'FAC': '設施',
 'GPE': '行政區',
 'LANGUAGE': '語言',
 'LAW': '法律',
 'LOC': '地理區',
 'MONEY': '金錢',
 'NORP': '民族、宗教、政治團體',
 'ORDINAL': '序數',
 'ORG': '組織',
 'PERCENT': '百分比率',
 'PERSON': '人物',
 'PRODUCT': '產品',
 'QUANTITY': '數量',
 'TIME': '時間',
 'WORK_OF_ART': '作品'}

In [57]:
com_df = pos_df.merge(ner_df, left_index=True, right_index=True)
com_df = com_df.applymap(combine_fn)
all_df = df2.merge(com_df, left_index=True, right_index=True)
all_df

,index,content,非謂形容詞,冒號,逗號,對等連接詞,連接詞，如：等等,關聯連接詞,副詞,的之得地,...,日期,行政區,語言,法律,金錢,序數,組織,百分比率,時間,作品
0,1,法規名稱：中華民國證券投資信託暨顧問商業同業公會「會員及其銷售機構從事廣告及營業活動行為規範...,信託,：、：,,暨、及、及,,,,,...,民國 109 年 01 月 07 日,,,,,,中華民國證券投資信託暨顧問商業同業公會,,,
1,2,第一章 總則第一條本行為規範依據證券投資顧問事業管理規則第十四條第二項、證券投資信託事業...,信託、信託,,,暨、及,,,,之,...,,,,,,第二、第二十二條、第六、第七、第一、第十四條、第二、第五十條、第一,境外基金,,,
2,3,第二條本公會會員及其銷售機構，為經營、辦理或推展證券投資信託、證券投資顧問、全權委託投資與境...,信託、信託,,，、，、，、，,及、或、與、及、及、及,,並,全權、應,,...,,,,,,第二,,,,
3,4,第三條證券投資信託事業委任之基金銷售機構從事證券投資信託基金之廣告、公開說明會及其他營業活動...,信託、信託、信託,,，、，,及,,,確實、不可、應,之、之、之,...,,,,,,第三,,,,
4,5,第四條總代理人委任之基金銷售機構從事境外基金之廣告、公開說明會及其他營業活動有違反本行為規範...,,,，、，、，、，、，,及、及、或、及、及,,但,應、得、確實、不可、應,之、之、之、之、之,...,,,,,,第四,,,,
5,6,第五條本行為規範所稱廣告，指以促進業務為目的，運用下列傳播媒體，就第二條業務及其相關事務為傳...,下列、公共,：,，、，、，、，、，,及、或、或、或、或、或、或、與、或、或、或、或,等、等、等,,所、所、所,之、之、之、之、之、之、之、之,...,,,,,,第五、第二,,,,
6,7,第六條本行為規範所稱公開說明會及其他營業活動，指為促進業務為目的，以舉辦現場講習會、座談會、...,一般、下列,：,，、，、，、，、，、，、，、，、，、，,及、或、或、以及、或、或、或、及、及、及、及、及、或、或、或、及、及,等,一、並、四,所、應、應、所、不得、所、應、應、得、應,之、之、之、之、之、之、之、之、之、之,...,,,中文,,,第一、第六,,,,
7,8,第二章 證券投資信託暨境外基金業務第七條證券投資信託事業、總代理人及其委任之基金銷售機構...,信託、信託、下列,：,，、，、，、，、，、，、，、，、，、，、，,暨、及、或、或、及、及,等,以、並、一、如、但、四、並、若,須、應、所、應、應、則、免、仍、應、應、應、確實、再、應、所、不得,之、之、之、之、之、之、之、之、得、之、之、之、之,...,,,,,,第二、第七,金管會、金管會,,,
8,9,第八條證券投資信託事業、總代理人及基金銷售機構從事基金之廣告、公開說明會及其他營業活動時，不...,信託、下列、信託、主要、開放式、主要、主要、淨、主要,：,，、，、，、，、，、，、，、，、，、，、，、，、，、，、，、，、，、，、，、，、，、，、，、...,及、及、或、或、或、或、或、或、及、或、或、或、或、或、及、或、或、或、與、或、或、或、或、...,等、等、等、等、等、等,一、但、但、四、六、十二、以、十八、但、以免、若,不得、已、能、已、未、預、可能、不、未、未、所、已、尚未、未、所、未、未、尚未、未、已、可、...,之、之、之、之、之、之、之、之、之、之、之、之、之、之、之、之、之、之、之、之、之、之、之、...,...,三十,,,,,第八,金管會、金管會、金管會、金管會、金管會,,,基金月報
9,10,第八條之一證券投資信託事業、總代理人及基金銷售機構從事基金業務之宣導推廣活動時，得在不與基金...,信託、下列、單一、信託、下列、單一、非、信託、信託,：、：、：,，、，、，、，、，、，、，、，、，、，、，、，、，、，、，、，、，,及、或、及、或、且、或、及、及、或、或、或,等、等、等,並、並、且、一、以、但、並、四、如、並,得、不、應、不得、變相、應、不得、重複、不得、應、確實、應、應、所、應、應、按月、應、可、應...,之、之、之、之、之、之、之、之、之、之、之,...,上個月份、上個月份、二年、每月十日前,,,,新臺幣二百元、新臺幣三十元,第一、第二、第四、第八,金管會,,,


In [58]:
all_df.to_excel('D:/python/LEO_TM/BERT_chinese_LM_processing/Results/paragraph_tok_ner_Leo.xlsx',index=False)